In [2]:
import pandas as pd
import numpy as np
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import Imputer
from sklearn import cross_validation
from sklearn import covariance
from sklearn import ensemble
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor as KNN
import string
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
import math
import string
import os
import datetime

In [3]:
train_data = pd.read_csv("train_imputed.csv")
train_data.head()

,tax_value,property_size,zestimate_amount,bathrooms,tax_year,latitude,bedrooms,year_built,home_size,longitude,...,precinct_string_26.0,precinct_string_28.0,precinct_string_30.0,precinct_string_32.0,precinct_string_33.0,precinct_string_34.0,precinct_string_5.0,precinct_string_6.0,precinct_string_7.0,precinct_string_9.0
0,2290000.0,2000.0,849427.0,1.0,2015.0,40.724448,1.0,1910.0,400.0,-73.980284,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1674000.0,2250.0,7332356.0,1.0,2015.0,40.724385,2.0,1900.0,10800.0,-73.980120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2290000.0,2000.0,1991289.0,1.0,2015.0,40.724332,4.0,1910.0,3374.0,-73.980007,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2837000.0,650.0,1225236.0,1.0,2015.0,40.724225,1.0,1876.0,650.0,-73.979720,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,6701000.0,2250.0,12785150.0,1.0,2015.0,40.724125,2.0,1901.0,4168.0,-73.979495,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [4]:
x_train = train_data.drop(['zestimate_amount'],1)
y_train = train_data['zestimate_amount']

In [6]:
x_train = train_data.drop(['zestimate_amount'],1)
y_train = train_data['zestimate_amount']
# Parameters for tuning
n_trees = np.arange(10, 100, 20)  # Trees and depth are explored on an exponentially growing space,
#depths = np.arange(20, 50)   # since it is assumed that trees and depth will add accuracy in a decaying fashion.
# To keep track of the best model
best_score = 0
# Run grid search for model with 5-fold cross validation
print '5-fold cross validation:'
for trees in n_trees:
        
    # Cross validation for every experiment
    k_folds = cross_validation.KFold(x_train.shape[0], n_folds=5, shuffle=True)
    scores = []
    for train_indices, validation_indices in k_folds:
        # Generate training data
        x_train_cv = x_train.iloc[train_indices].values
        y_train_cv = y_train[train_indices].values
        # Generate validation data
        x_validate = x_train.iloc[validation_indices].values
        y_validate = y_train[validation_indices].values
            
        # Fit random forest on training data
        model = ensemble.RandomForestClassifier(n_estimators=trees, max_depth=20)
        model.fit(x_train_cv, y_train_cv)
        # Score on validation data
        scores += [model.score(x_validate, y_validate)]
        
    # Record and report accuracy
    average_score = np.mean(scores)
    print "Trees:", trees, "Depth:", 20, "Score:", average_score
        
    # Update our record of the best parameters see so far
    if average_score > best_score:
        best_score = average_score
        best_trees = trees
            
# Fit model on entire train set using chosen number of trees and depth
model = ensemble.RandomForestRegressor(n_estimators=best_trees, max_depth=20)
model.fit(x_train, y_train)
print 'Chosen number of trees, depth:', best_trees, ',', best_depth
print 'Test accuracy:', model.score(x_test, y_test)

 5-fold cross validation:
Trees: 10 Depth: 20 Score: 0.015137725922
Trees: 30 Depth: 20 Score: 0.0219119955731
Trees: 50 Depth: 20 Score: 0.0260350477301
Trees: 70 Depth: 20 Score: 0.0262118476038
Trees: 90 Depth: 20 Score: 0.0278024913656
Chosen number of trees, depth: 90 ,

NameError: name 'best_depth' is not defined

In [ ]:
# Parameters for tuning
n_trees = np.arange(10, 100, 20)  # Trees and depth are explored on an exponentially growing space,
depths = np.arange(2, 10)   # since it is assumed that trees and depth will add accuracy in a decaying fashion.
# To keep track of the best model
best_score = 0
# Run grid search for model with 5-fold cross validation
print '5-fold cross validation:'
for trees in n_trees:
    for depth in depths:
        
        # Cross validation for every experiment
        k_folds = cross_validation.KFold(x_train.shape[0], n_folds=5, shuffle=True)
        scores = []
        for train_indices, validation_indices in k_folds:
            # Generate training data
            x_train_cv = x_train.iloc[train_indices].values
            y_train_cv = y_train[train_indices].values
            # Generate validation data
            x_validate = x_train.iloc[validation_indices].values
            y_validate = y_train[validation_indices].values
            
            # Fit random forest on training data
            model = ensemble.RandomForestClassifier(n_estimators=trees, max_depth=depth)
            model.fit(x_train_cv, y_train_cv)
            # Score on validation data
            scores += [model.score(x_validate, y_validate)]
        
        # Record and report accuracy
        average_score = np.mean(scores)
        print "Trees:", trees, "Depth:", depth, "Score:", average_score
        
        # Update our record of the best parameters see so far
        if average_score > best_score:
            best_score = average_score
            best_trees = trees
            best_depth = depth
            
# Fit model on entire train set using chosen number of trees and depth
model = ensemble.RandomForestRegressor(n_estimators=best_trees, max_depth=best_depth)
model.fit(x_train, y_train)
print 'Chosen number of trees, depth:', best_trees, ',', best_depth
print 'Test accuracy:', model.score(x_test, y_test)


5-fold cross validation:


In [ ]:
# Parameters for tuning
n_trees = np.arange(10, 100, 20)  # Trees and depth are explored on an exponentially growing space,
#depths = np.arange(20, 50)   # since it is assumed that trees and depth will add accuracy in a decaying fashion.
# To keep track of the best model
best_score = 0
# Run grid search for model with 5-fold cross validation
print '5-fold cross validation:'
for trees in n_trees:
    # Cross validation for every experiment
    k_folds = cross_validation.KFold(x_train.shape[0], n_folds=5, shuffle=True)
    scores = []
    for train_indices, validation_indices in k_folds:
        # Generate training data
        x_train_cv = x_train.iloc[train_indices].values
        y_train_cv = y_train[train_indices].values
        # Generate validation data
        x_validate = x_train.iloc[validation_indices].values
        y_validate = y_train[validation_indices].values
            
        # Fit random forest on training data
        model = ensemble.RandomForestClassifier(n_estimators=trees, max_depth=20)
        model.fit(x_train_cv, y_train_cv)
        # Score on validation data
        scores += [model.score(x_validate, y_validate)]
        
    # Record and report accuracy
    average_score = np.mean(scores)
    print "Trees:", trees, "Depth:", depth, "Score:", average_score
        
    # Update our record of the best parameters see so far
    if average_score > best_score:
            best_score = average_score
            best_trees = trees
            best_depth = depth
            
# Fit model on entire train set using chosen number of trees and depth
model = ensemble.RandomForestRegressor(n_estimators=best_trees, max_depth=best_depth)
model.fit(x_train, y_train)
print 'Chosen number of trees, depth:', best_trees, ',', best_depth
print 'Test accuracy:', model.score(x_test, y_test)